## EXTRACCIÓN DE CARACTERÍSTICAS
Este notebook extrae glosarios para las distintas categorías de texto utilizando la librería gensim.

In [41]:
!pip3 install gensim 

In [42]:
import pandas as pd
import os
import gensim
from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

Cargamos nuestro dataframe de noticias dedicado al entrenamiento

In [43]:
noticias_dataframe = pd.read_csv('../Datos/noticias_train.csv')
noticias_dataframe_t = pd.read_csv('../Datos/noticias_test.csv')

Separamos nuestras noticias en diferentes cojuntos de datos por categoría de noticia

In [44]:
deportes_dataframe = noticias_dataframe[noticias_dataframe["category"]=="sports"]
deportes_dataframe.info()

print("-"*60)

deportes_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="sports"]
deportes_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 

In [45]:
salud_dataframe = noticias_dataframe[noticias_dataframe["category"]=="health"]
salud_dataframe.info()

print("-"*60)

salud_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="health"]
salud_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object

In [46]:
ciencia_dataframe = noticias_dataframe[noticias_dataframe["category"]=="science"]
ciencia_dataframe.info()

print("-"*60)

ciencia_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="science"]
ciencia_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 60 to 89
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 60 to 89
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object

In [47]:
politica_dataframe = noticias_dataframe[noticias_dataframe["category"]=="politics"]
politica_dataframe.info()

print("-"*60)

politica_dataframe_t = noticias_dataframe_t[noticias_dataframe_t["category"]=="politics"]
politica_dataframe_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 90 to 119
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 90 to 119
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     obje

### Extracción de glosario mediante TFIDF


La forma más sencilla para obtener nuestro glosario es mediante el esquema *tf-idf*

In [48]:
def extrae_glosario_tf_idf(categoria_dataframe, size):
    """ 
    Funcion para extraer el glosario

    - categoria_dataframe: cojunto de datos de una categoriad de noticias
    - size: tokens del glosario a mostrar
    """
    # Pre-procesamos el corpus de cada noticia
    doc_tokens = [simple_preprocess(corpus) for corpus in categoria_dataframe["corpus"]]

    # Hacemos un diccionario. Mapeo de palabras e identificadores de palabra
    dictionary = corpora.Dictionary(doc_tokens)

    # Convertimos el diccionario al formato BoW
    bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokens]

    # https://radimrehurek.com/gensim/models/tfidfmodel.html
    # Implementamos un modelo tf-idf
    tfidf = models.TfidfModel(bow_corpus, smartirs="lfc")
    bow_tfidf = tfidf[bow_corpus]

    # Por cada documento, sacamos un glosario
    tfidf_dic = {dictionary.get(id): value for doc in bow_tfidf for id, value in doc}
    tfidf_list = [k for k, v in sorted(tfidf_dic.items(), key=lambda item: item[1], reverse = True)]

    return tfidf_list[:size]
    

In [49]:
glosarios = {'deportes': [],
             'salud': [],
             'ciencia': [],
             'salud': []}

glosarios_t = {'deportes': [],
            'salud': [],
            'ciencia': [],
            'salud': []}

In [50]:
glosarios['deportes'] = extrae_glosario_tf_idf(deportes_dataframe, 50)
print(glosarios['deportes'])
print(len(glosarios['deportes']))

print("-"*60)

glosarios_t['deportes'] = extrae_glosario_tf_idf(deportes_dataframe_t, 50)
print(glosarios_t['deportes'])
print(len(glosarios_t['deportes']))


['falso', 'jamas', 'smash', 'mclaren', 'kyrgios', 'tenista', 'ruud', 'respuesta', 'gigante', 'reserva', 'real', 'hernanga', 'mez', 'cuento', 'domenicali', 'chicago', 'ferrero', 'fifa', 'butler', 'formato', 'booker', 'tatum', 'tenerife', 'campazzo', 'descalabro', 'alcaraz', 'juancho', 'magnussen', 'steiner', 'gavi', 'porra', 'gonzalez', 'gonzalo', 'kosmos', 'rotacion', 'gaal', 'austin', 'indycar', 'palou', 'enrique', 'arbitro', 'tristeza', 'uruguayo', 'hijo', 'lesia', 'toronto', 'simplemente', 'rosa', 'cd', 'krack']
50
------------------------------------------------------------
['seguidor', 'mbappe', 'warren', 'doncic', 'estabilidad', 'horford', 'shapovalov', 'cristiano', 'exencion', 'correr', 'carpena', 'enrique', 'luis', 'mans', 'stakhovsky', 'malaga', 'medico', 'booker', 'aliassime', 'auger', 'pts', 'reb', 'nets', 'cantidad', 'juventus', 'ronaldo', 'clippers', 'lebron', 'garden', 'florentino', 'heredero', 'gasto', 'atleta', 'colombia', 'roger', 'verdasco', 'dificil', 'dolares', 'ktm

In [51]:
glosarios['salud'] = extrae_glosario_tf_idf(salud_dataframe, 50)
print(glosarios['salud'])
print(len(glosarios['salud']))

print("-"*60)

glosarios_t['salud'] = extrae_glosario_tf_idf(salud_dataframe_t, 50)
print(glosarios_t['salud'])
print(len(glosarios_t['salud']))

['cafe', 'gripe', 'zumo', 'dash', 'genital', 'orina', 'vrs', 'unicef', 'cancer', 'verruga', 'una', 'pulmonar', 'automatico', 'resfriado', 'vision', 'pet', 'cannabis', 'huevo', 'esconder', 'tc', 'afiliado', 'eps', 'ips', 'creatividad', 'research', 'signo', 'uk', 'vejiga', 'condiloma', 'rehabilitacia', 'alga', 'wakame', 'morder', 'ansiedad', 'coosalud', 'mexico', 'lavar', 'padre', 'recoletas', 'estancamiento', 'mbst', 'esta', 'oms', 'fibrotico', 'vph', 'gen', 'macula', 'oseo', 'vegetal', 'viruela']
50
------------------------------------------------------------
['arterial', 'strep', 'antidepresivo', 'bronquiolitis', 'aprovechar', 'temporada', 'folcodina', 'depresion', 'cafa', 'mascarilla', 'aceite', 'oliva', 'entrenar', 'cabello', 'estigma', 'epilepsia', 'quiraorgica', 'congelado', 'cuadro', 'gotlib', 'marfan', 'manada', 'parasito', 'colapsar', 'intensivo', 'moralesla', 'padres', 'torre', 'pez', 'comida', 'mascaras', 'permafrost', 'ir', 'poveda', 'amigdala', 'hipocampo', 'tpbm', 'viatris

In [52]:
glosarios['politica'] = extrae_glosario_tf_idf(politica_dataframe, 50)
print(glosarios['politica'])
print(len(glosarios['politica']))

print("-"*60)

glosarios_t['politica'] = extrae_glosario_tf_idf(politica_dataframe_t, 50)
print(glosarios_t['politica'])
print(len(glosarios_t['politica']))

['silva', 'refera', 'torra', 'esparza', 'benito', 'sinema', 'ilacito', 'nicolas', 'isabel', 'nduma', 'almeida', 'demarcacia', 'ayres', 'mexicano', 'rufian', 'autoritariaa', 'cgp', 'adjudicacia', 'obrador', 'vicepresidencia', 'exfuncionario', 'arizona', 'ahorro', 'meloni', 'separado', 'juarez', 'agresion', 'delincuente', 'independiente', 'senador', 'electricidad', 'larsen', 'junts', 'catalunya', 'ndum', 'anunciado', 'gamarra', 'renovacia', 'smith', 'escuchas', 'mitsotakis', 'ebrard', 'embajada', 'estafa', 'falsedad', 'ana', 'lara', 'paases', 'juan', 'lgtbi']
50
------------------------------------------------------------
['rosell', 'vuelo', 'insulto', 'junts', 'precio', 'renovar', 'reyes', 'rufian', 'consulta', 'anunciara', 'sandro', 'monta', 'multilateral', 'turismo', 'partida', 'preguntar', 'ucraniano', 'ndum', 'refera', 'nombramiento', 'ribera', 'falso', 'juventud', 'podem', 'unilateral', 'urgente', 'educativo', 'auxiliar', 'lamite', 'normalidad', 'desbloqueo', 'ministros', 'designac

In [53]:
glosarios['ciencia'] = extrae_glosario_tf_idf(ciencia_dataframe, 50)
print(glosarios['ciencia'])
print(len(glosarios['ciencia']))

print("-"*60)

glosarios_t['ciencia'] = extrae_glosario_tf_idf(ciencia_dataframe_t, 50)
print(glosarios_t['ciencia'])
print(len(glosarios_t['ciencia']))


['idioma', 'llnl', 'llama', 'sal', 'grb', 'sapiens', 'cuerpo', 'congelacia', 'antena', 'banyoles', 'neandertal', 'ofensivo', 'palabrota', 'cernan', 'ska', 'canaria', 'congelar', 'bankman', 'fried', 'estruendo', 'programador', 'ftx', 'york', 'fuego', 'sonido', 'isidro', 'menta', 'radiotelescopio', 'estallido', 'trastorno', 'alphacode', 'ba', 'gamma', 'nebulosa', 'smico', 'canarias', 'isla', 'mutacion', 'calificar', 'fonema', 'taco', 'deepmind', 'distrito', 'imputar', 'santo', 'calcio', 'descender', 'puente', 'mandabula', 'martanez']
50
------------------------------------------------------------
['vacuna', 'invierno', 'gripe', 'navidad', 'cuantico', 'captura', 'honor', 'fusia', 'cola', 'fraa', 'gemanidas', 'alcohol', 'latigo', 'supersa', 'regalo', 'tabaco', 'quipus', 'agujero', 'apple', 'antageno', 'subtipo', 'hemofilia', 'estampido', 'sonido', 'ue', 'km', 'fasica', 'galaxia', 'virus', 'magic', 'reencuentro', 'alineacia', 'almanaque', 'escarcha', 'helada', 'tv', 'barrera', 'mola', 'pers

Guardamos los glosarios en .txt

In [54]:
if os.path.exists("../Datos/Glosarios") == False:
    os.mkdir("../Datos/Glosarios")
    
if os.path.exists("../Datos/Glosarios/train") == False:
    os.mkdir("../Datos/Glosarios/train")

if os.path.exists("../Datos/Glosarios/test") == False:
    os.mkdir("../Datos/Glosarios/test")


In [55]:
inicial = "../Datos/Glosarios/train/glosario"

for g in glosarios.items():
    tipo = g[0]
    fname = f"{inicial}_{tipo}.txt"
    with open(fname, 'w') as f:
        for termino in g[1]:
            f.write(termino)
            f.write('\n')

In [56]:
inicial = "../Datos/Glosarios/test/glosario"
for g in glosarios_t.items():
    tipo = g[0]
    fname = f"{inicial}_{tipo}.txt"
    with open(fname, 'w') as f:
        for termino in g[1]:
            f.write(termino)
            f.write('\n')
        